In [1]:
!pip3 install tweepy textblob
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

In [2]:
%load_ext dotenv
%dotenv ./auth.txt

In [3]:


class TwitterClient(object):
    def __init__(self):
        try:
            self.auth = OAuthHandler(os.getenv('api_key'), os.getenv('api_secret'))
            self.auth.set_access_token(os.getenv('oauth_token'), os.getenv('oauth_token_secret'))
            self.api = tweepy.API(self.auth)
            assert self.api
        except:
            print("Error: Authentication Failed")
    
    
    def get_tweets(self, query, count = 10):
        tweets = []
        try:
            fetched_tweets = self.api.search_tweets(q = query, count = count)
            for tweet in fetched_tweets:
                parsed_tweet = {}
                parsed_tweet['text'] = tweet.text
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
                if tweet.retweet_count > 0:
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
            return tweets
        except tweepy.TweepyException as e:
            print("Error : " + str(e))

    def fetch_tweets(self, query, count = 10):
        try:
            return self.api.search_tweets(q = query, count = count)
        except tweepy.TweepyException as e:
            print("Error : " + str(e))  

In [4]:
tc = TwitterClient()
#tc.fetch_tweets('#FarmLaws',1)

In [35]:
## The tweets have been fetched and stored in a file. 
## Use the cached tweets for consistent results instead of fetching new from twitter.
with open('farmer.txt', 'w') as f:
    tweets = tc.fetch_tweets('#FarmLaws',8000)
    for tweet in tweets:
        f.write(tweet.text + "_$_")

# 1. Data Preparation

## 1.1 Fetch from file

In [147]:

f = open('farmer.txt', 'r')
tweets = f.read().split('_$_')
f.close()

In [148]:
tweets[0]

'RT @Jasvenkaur: #FarmLaws Solution @mssirsa Is \n#KhalistanReferendum NOT @BJP4India @TajinderBagga \n#SFJ Dares @JPNadda @AmitShah @mssirsa…'

# 1.2 Preprocess each tweet

In [149]:
import re
import string

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk import download
# download('stopwords')

def process_tweet(tweet):
    '''
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    '''
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
            word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    tweets_eng = []
    for word in tweets_clean:
        flag = True
        for i in word:
            if ord(i) >= 256:
                flag = False
                break
        if flag:
            tweets_eng.append(word)
    return tweets_eng

In [150]:
processed_tweets = [process_tweet(tweet) for tweet in tweets]
print(tweets[0])
print(processed_tweets[0])

RT @Jasvenkaur: #FarmLaws Solution @mssirsa Is 
#KhalistanReferendum NOT @BJP4India @TajinderBagga 
#SFJ Dares @JPNadda @AmitShah @mssirsa…
['farmlaw', 'solut', 'khalistanreferendum', 'sfj', 'dare']


# 2. Feature Extraction

## 2.1 Word counts  

In [151]:
def count_words(tweet:list, freqs:dict, wordToTweet:dict):
    for word in tweet:
        if word in freqs:
            freqs[word] += 1
            wordToTweet[word].append(tweet)
        else:
            freqs[word] = 1
            wordToTweet[word] = [tweet]
    return freqs

In [152]:
freqs = dict()
wordToTweet = dict()
for tweet in processed_tweets:
    count_words(tweet, freqs, wordToTweet)

In [153]:
max(freqs.values())

80

# 3. Analysis

In [154]:
#!pip3 install numpy pandas seaborn
import numpy as np
import pandas as pd
import seaborn as sb

In [155]:
# freqs.keys()
# list(freqs.items())

In [156]:
len(freqs.keys())

103

In [157]:
most_freq_word = np.argmax(list(freqs.values()))
list(freqs.items())[most_freq_word]

('farmlaw', 80)

In [158]:

# Sorted in descending order of frequencies
freq_sorted = list(freqs.items())
freq_sorted.sort(key = lambda x : -x[1])
freq_sorted[:10]

[('farmlaw', 80),
 ('sfj', 72),
 ('solut', 61),
 ('khalistanreferendum', 60),
 ('dare', 43),
 ('account', 23),
 ('bjp', 16),
 ('sirsa', 15),
 ('law', 12),
 ('intern', 11)]

# Clustering

In [159]:
def vectorize(tweet, alphabet):
    v = np.zeros(len(alphabet))
    for i in range(len(alphabet)):
        if alphabet[i] in tweet:
            v[i] += 1
    return v

def normalize(v):
    norm = np.power(np.sum(np.power(v,2)), 0.5)
    if norm == 0:
        return 0
    return v / norm

In [160]:
from collections import Counter
def len_counts(clusters):
    lens = [len(cluster) for cluster in clusters.values()]
    return dict(Counter(lens))

In [161]:
def get_avg_dist(clusterA, clusterB, vectors):
    dist_sum = 0
    for i in clusterA:
        for j in clusterB:
            dist_sum += np.sum(np.power(vectors[i] - vectors[j], 2))
    
    dist_avg = dist_sum / (len(clusterA) * len(clusterB))
    return dist_avg

In [165]:
def mergeOnce(clusters, vectors):
    min_dist = 2**30
    min_loc = (0, 0)
    for clusterInxA in clusters:
        for clusterInxB in clusters:
            if clusterInxA != clusterInxB:
                dist = get_avg_dist(clusters[clusterInxA], clusters[clusterInxB], vectors)
                if dist < min_dist:
                    min_loc = (clusterInxA, clusterInxB)
                    min_dist = dist

    clusters[min_loc[0]] = clusters[min_loc[0]] + clusters[min_loc[1]]
    clusters.pop(min_loc[1])

def mergeToK(clusters, vectors, K):
    while len(clusters.keys()) > K:
        mergeOnce(clusters, vectors)

In [166]:
def display_unique_tweets(tweets, cluster):
    c_tweets = [tweets[i] for i in cluster]
    for i in range(len(c_tweets)):
        if not c_tweets[i] in c_tweets[:i]:
            print(c_tweets[i])
        

In [172]:
alphabet = [i[0] for i in freq_sorted]
n = len(processed_tweets)
m = len(alphabet)
clusters = dict()
vectors = dict()
for i in range(n):
    vectors[i] = vectorize(processed_tweets[i], alphabet)
    clusters[i] = [i]
mergeToK(clusters, vectors, 15)


In [176]:
print("lengths:")
print({cluster:len(clusters[cluster]) for cluster in clusters})
print("len counts:")
print(len_counts(clusters))

lengths:
{0: 53, 3: 4, 5: 3, 6: 1, 15: 8, 16: 6, 19: 1, 31: 1, 36: 3, 41: 1, 43: 8, 44: 1, 47: 2, 48: 1, 60: 1}
len counts:
{53: 1, 4: 1, 3: 2, 1: 7, 8: 2, 6: 1, 2: 1}


In [192]:
display_unique_tweets(processed_tweets, clusters[60])

['sfj', 'make', 'account', 'sirsa', 'new', 'fellow', 'bjp', 'farmlaw', 'khalistanreferendum', 'sfj']
